In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, window

In [2]:
sparkSession = SparkSession \
    .builder \
    .appName("TwitterStreamingAssignment") \
    .master("local") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0") \
    .getOrCreate()

22/01/11 17:11:09 WARN Utils: Your hostname, EMPID21092 resolves to a loopback address: 127.0.1.1; using 192.168.43.37 instead (on interface wlp3s0)
22/01/11 17:11:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/rita/.ivy2/cache
The jars for the packages stored in: /home/rita/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2f417c77-8f27-46aa-992b-26ac982290e6;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in local-m2-cache
	found com.google.code.findbugs#jsr305;3.0.0 in cent

In [3]:
twitterDataDF = sparkSession.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "trump") \
    .option("startingOffsets", "earliest") \
    .load()

In [4]:
twitterDataDF = twitterDataDF.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "CAST(timestamp AS TIMESTAMP)")

print(twitterDataDF)

DataFrame[key: string, value: string, timestamp: timestamp]


In [5]:
words = twitterDataDF\
    .select(explode(split(twitterDataDF.value, " ")).alias("word"),twitterDataDF["timestamp"])

In [6]:
count = words.select(words['word'],words['timestamp'])\
    .filter(words["word"] == "omicron")\
    .groupBy(window(words["timestamp"],"2 minutes"),words["word"])\
    .count()

In [7]:
query = count.writeStream.outputMode("complete").format("console").start()

22/01/11 17:11:15 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-2e9dcd95-f2e0-4222-96af-4f58a5fe045e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/01/11 17:11:15 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
